In [3]:
#!/usr/bin/env python
# coding: utf-8

import shutil
import sys
import glob
import tarfile
import os
import mne
import pandas as pd
import numpy as np
from tqdm import tqdm
from directory_tree import display_tree
from mne_connectivity import spectral_connectivity_epochs
from mne.datasets import sample
import numpy as np
from collections import Counter
import ast

def ReadFile():
    EEG = []
    MetaData = pd.read_csv('/workspace/brail_poc3/experiments/Behavioural_Data_MPILMBB_LEMON/META_File_IDs_Age_Gender_Education_Drug_Smoke_SKID_LEMON.csv')

    # file list over 40
#     fileList = [item for item in sorted(glob.glob("/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/*")) if item.split('/')[-1] in ground_truth["ID"].tolist()]
    # file list all
    fileList = sorted(glob.glob("/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/*"))

    for path in tqdm(fileList):
        print("Subject number:", path.split('/')[-1])
        Subjects = {}
        Subjects["Subject_Number"] = path.split('/')[-1]
        Subjects["prepSignal_EC"] = mne.io.read_raw_eeglab("/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/{}/{}/{}_EC.set".format(Subjects["Subject_Number"], Subjects["Subject_Number"], Subjects["Subject_Number"]), preload=True, verbose=False)
        Subjects["prepSignal_EC"] = InterpolateCh(Subjects["Subject_Number"], Subjects["prepSignal_EC"])

#         Subjects["prepSignal_EO"] = mne.io.read_raw_eeglab("/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/{}/{}/{}_EO.set".format(Subjects["Subject_Number"], Subjects["Subject_Number"], Subjects["Subject_Number"]), preload=True, verbose=False)
#         Subjects["prepSignal_EO"] = InterpolateCh(Subjects["Subject_Number"], Subjects["prepSignal_EO"])

        Subjects["Age"] = MetaData[MetaData['Subject'] == Subjects["Subject_Number"]].iloc[0,2]
        Subjects["Age_Number"] = int(Subjects["Age"].split('-')[0])
        EEG.append(Subjects)
    return EEG

def CauConn(ChanNum, data, metrics):
    len_channel = data.shape[1]
    index1 = [ChanNum] * (len_channel-1)
    index2 = list(np.arange(0, len_channel, 1))
    del index2[ChanNum]
    indices = (index1, index2)
    print(indices)
    conn = spectral_connectivity_epochs(
            data, method=metrics, sfreq=250, indices=indices,
            fmin=9, fmax=11, faverage=True, verbose=False).get_data()[:, 0]
    conn = conn.tolist()
    return conn

def splitEEG(prepEEG):
#     epochs = []
#     ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])
#     annotation_EC_df = prepEEG.annotations.to_data_frame()
#     EC_sec = annotation_EC_df[annotation_EC_df.columns[0]].dt.strftime('%s.%f').astype(float) + 32400
#     point_indices = annotation_EC_df.index[annotation_EC_df['description'] == 'boundary']
#     BreakPoint = []
# #     print(EC_sec[point_indices])
#     for i, j in enumerate(EC_sec[point_indices]):
#         if i == 0:
#             BreakPoint.append([0, int(j)])
#         else:
#             BreakPoint.append([BreakPoint[-1][1], int(j)])
#         if i == len(EC_sec[point_indices])-1:
#             BreakPoint.append([int(j)+1, int(EC_sec.iloc[-1])])
# #     print(BreakPoint)
#     for i,d  in enumerate(BreakPoint): # Ignore the short duration between each braek point.
#         if d[1] - d[0] < 15:
#             del BreakPoint[i]
#     for i, d in enumerate(BreakPoint):  # Splite each epoch by break point
#         print("The {} trial from {} to {}. ({}sec)".format(ordinal(i+1), d[0], d[1], d[1]-d[0]))
    events = mne.make_fixed_length_events(prepEEG.copy(), duration=2, overlap=0.2)
    epochs = mne.Epochs(prepEEG, events=events, event_id=None, tmin=0, tmax=2, baseline=None, preload=False, verbose=False)
#         epochs.append(prepEEG.copy().crop(tmin=d[0], tmax=d[1]))
#         print("------------------------------------------------")
    return epochs

def EpochRotate(OneEpoch, metric):
    Chan_PLI = []
    data = OneEpoch.get_data()
    print(data.shape)
    for i in range(data.shape[1]):
        Chan_PLI.append(CauConn(i, data, metric))
    for i in range(data.shape[1]):
        Chan_PLI[i].insert(i, 0)
    return np.array(Chan_PLI)

def InsertCh(Ch_name, raw):
    # Need NoneDel_prepSignal to get local info
    # Used for InterpolateCh

    prep_len = raw._data.shape[1]
#     new_channel_chs = raw.info['chs'][-1]
#     new_channel_chs['ch_name'] = Ch_name
#     new_channel_chs['scanno'] = new_channel_chs['scanno']+1
#     new_channel_chs['logno'] = new_channel_chs['logno']+1
#     new_channel_chs['loc'] = [item.get('loc') for item in NoneDel_prepSignal.info['chs'] if(item['ch_name'] == Ch_name)][0]
#     print(new_channel_chs)

    new_channel_data = np.array([0] * prep_len)
    new_channel_info = mne.create_info(ch_names=[Ch_name], sfreq=raw.info['sfreq'], ch_types=['eeg'])
    new_channel_info['chs'][0]['loc'] = [item.get('loc') for item in NoneDel_prepSignal.info['chs'] if(item['ch_name'] == Ch_name)][0]
    print(new_channel_info['chs'])
    new_channel_data = new_channel_data[np.newaxis, :]  # 新しいチャンネルデータを適切な形式に変換
    new_raw = mne.io.RawArray(new_channel_data, new_channel_info)
    raw.add_channels([new_raw], force_update_info=True)
    return raw

def InterpolateCh(ID, raw):
    # Target EC, Need del_ch file
    # Interpolate missing channel in preprocessed LEMON EEG data

    CH = del_ch[del_ch["ID"] == ID]['del_channel']
    del_Ch_list = ast.literal_eval(CH.tolist()[0])
    prepSignal_EC_inte = raw.copy()

    for i in [item for item in del_Ch_list if item != 'VEOG']:
        prepSignal_EC_inte = InsertCh(i, prepSignal_EC_inte)
        prepSignal_EC_inte.info["bads"].append(i)

    prepSignal_EC_inte.info

    # Interpolate channels
    prepSignal_EC_inte = prepSignal_EC_inte.interpolate_bads()

    # Reorder Channel
    Ch_order = NoneDel_prepSignal.ch_names
    prepSignal_EC_inte.reorder_channels(Ch_order)

    return prepSignal_EC_inte

def convert_age_range(age_range):
    return int(age_range.split("-")[0])


In [4]:
#################################
#                               #
# Example:                      #
# >> python3 FC_EC_Cal.py coh   #
#                               #
#################################

# Initial variables
NoneDel_prepSignal = NoneDel_prepSignal = mne.io.read_raw_eeglab('/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/sub-032303/sub-032303/sub-032303_EC.set', preload=True)
del_ch = pd.read_csv("/workspace/brail_poc3/notebooks/Chen/Sub_channel_list.csv", index_col=0)
Cog_score = pd.read_csv("/workspace/brail_poc3/notebooks/Chen/TestCognitive.csv")
Cog_score = Cog_score.loc[:, ["ID", "Label"]]
Age_data = pd.read_csv("/workspace/brail_poc3/experiments/Behavioural_Data_MPILMBB_LEMON/META_File_IDs_Age_Gender_Education_Drug_Smoke_SKID_LEMON.csv")
Age_data = Age_data.loc[:, ["Subject", "Age"]]
Age_data = Age_data.rename({'Subject': 'ID'}, axis='columns')
Age_data["Age_1"] = Age_data["Age"].apply(convert_age_range)
Age_data.drop("Age", axis=1, inplace=True)
cog_df = Cog_score.merge(Age_data, on='ID', how='left')
ground_truth = cog_df[cog_df['Age_1'] >= 40]



Reading /workspace/brail_poc3/experiments/Lemon_PreEEGLAB/sub-032303/sub-032303/sub-032303_EC.fdt
Reading 0 ... 119277  =      0.000 ...   477.108 secs...


/tmp/ipykernel_336521/3516541724.py:9: RuntimeWarning: Data file name in EEG.data (sub-010004_EC.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-032303_EC.fdt).
  NoneDel_prepSignal = NoneDel_prepSignal = mne.io.read_raw_eeglab('/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/sub-032303/sub-032303/sub-032303_EC.set', preload=True)
/tmp/ipykernel_336521/3516541724.py:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  NoneDel_prepSignal = NoneDel_prepSignal = mne.io.read_raw_eeglab('/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/sub-032303/sub-032303/sub-032303_EC.set', preload=True)
/tmp/ipykernel_336521/3516541724.py:9: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  NoneDel_prepSignal = NoneDel_prepSignal = mne.io.read_raw_eeglab('/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/sub-032303/sub-032303

In [5]:
ground_truth

,ID,Label,Age_1
7,sub-032383,2,60
13,sub-032318,2,60
33,sub-032331,1,70
34,sub-032305,1,65
35,sub-032308,1,60
...,...,...,...
161,sub-032495,1,70
162,sub-032497,1,60
163,sub-032498,1,65
164,sub-032496,2,60


In [6]:
# Read file in dict
print("------------------------------------------------")
print("Read file in dict")
EEG = ReadFile()
print(EEG)

------------------------------------------------
Read file in dict


  0%|                                                    | 0/61 [00:00<?, ?it/s]

Subject number: sub-032301
[{'loc': array([-8.45385386e-02,  5.17649254e-18, -8.84508251e-03,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,             nan,             nan,
                   nan,             nan,             nan,             nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'ch_name': 'T7', 'scanno': 1, 'logno': 1}]
Creating RawArray with float64 data, n_channels=1, n_times=119345
    Range : 0 ... 119344 =      0.000 ...   477.376 secs


/tmp/ipykernel_336521/1285187122.py:33: RuntimeWarning: Data file name in EEG.data (sub-010002_EC.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-032301_EC.fdt).
  Subjects["prepSignal_EC"] = mne.io.read_raw_eeglab("/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/{}/{}/{}_EC.set".format(Subjects["Subject_Number"], Subjects["Subject_Number"], Subjects["Subject_Number"]), preload=True, verbose=False)
/tmp/ipykernel_336521/1285187122.py:33: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  Subjects["prepSignal_EC"] = mne.io.read_raw_eeglab("/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/{}/{}/{}_EC.set".format(Subjects["Subject_Number"], Subjects["Subject_Number"], Subjects["Subject_Number"]), preload=True, verbose=False)
/tmp/ipykernel_336521/1285187122.py:33: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.


Ready.
[{'loc': array([-0.08041001, -0.0262075 , -0.0085086 ,  0.        ,  0.        ,
        0.        ,         nan,         nan,         nan,         nan,
               nan,         nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'ch_name': 'TP7', 'scanno': 1, 'logno': 1}]
Creating RawArray with float64 data, n_channels=1, n_times=119345
    Range : 0 ... 119344 =      0.000 ...   477.376 secs
Ready.
Interpolating bad channels
    Automatic origin fit: head of radius 85.0 mm
Computing interpolation matrix from 59 sensor positions
Interpolating 2 sensors


...

Subject number: sub-032498
[{'loc': array([ 0.02613301,  0.08078401, -0.00400108,  0.        ,  0.        ,
        0.        ,         nan,         nan,         nan,         nan,
               nan,         nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'ch_name': 'Fp2', 'scanno': 1, 'logno': 1}]
Creating RawArray with float64 data, n_channels=1, n_times=119977
    Range : 0 ... 119976 =      0.000 ...   479.904 secs
Ready.


/tmp/ipykernel_336521/1285187122.py:33: RuntimeWarning: Data file name in EEG.data (sub-010291_EC.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-032498_EC.fdt).
  Subjects["prepSignal_EC"] = mne.io.read_raw_eeglab("/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/{}/{}/{}_EC.set".format(Subjects["Subject_Number"], Subjects["Subject_Number"], Subjects["Subject_Number"]), preload=True, verbose=False)
/tmp/ipykernel_336521/1285187122.py:33: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  Subjects["prepSignal_EC"] = mne.io.read_raw_eeglab("/workspace/brail_poc3/experiments/Lemon_PreEEGLAB/{}/{}/{}_EC.set".format(Subjects["Subject_Number"], Subjects["Subject_Number"], Subjects["Subject_Number"]), preload=True, verbose=False)
/tmp/ipykernel_336521/1285187122.py:33: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.


[{'loc': array([-0.06842334,  0.04987138, -0.00748952,  0.        ,  0.        ,
        0.        ,         nan,         nan,         nan,         nan,
               nan,         nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'ch_name': 'F7', 'scanno': 1, 'logno': 1}]
Creating RawArray with float64 data, n_channels=1, n_times=119977
    Range : 0 ... 119976 =      0.000 ...   479.904 secs
Ready.
[{'loc': array([ 0.05927498, -0.03095528,  0.0524714 ,  0.        ,  0.        ,
        0.        ,         nan,         nan,         nan,         nan,
               nan,         nan]), 'unit_mul': 0 (FIFF_UNITM_NONE), 'range': 1.0, 'cal': 1.0, 'kind': 2 (FIFFV_EEG_CH), 'coil_type': 1 (FIFFV_COIL_EEG), 'unit': 107 (FIFF_UNIT_V), 'coord_frame': 4 (FIFFV_COORD_HEAD), 'ch_name': 'CP4', 'scanno': 1, 'logno': 1}]
Creating RawArray with float64 data, n_chan

100%|███████████████████████████████████████████| 61/61 [00:17<00:00,  3.40it/s]

[{'Subject_Number': 'sub-032301', 'prepSignal_EC': <RawEEGLAB | sub-032301_EC.fdt, 61 x 119345 (477.4 s), ~55.6 MB, data loaded>, 'Age': '65-70', 'Age_Number': 65}, {'Subject_Number': 'sub-032303', 'prepSignal_EC': <RawEEGLAB | sub-032303_EC.fdt, 61 x 119278 (477.1 s), ~55.6 MB, data loaded>, 'Age': '65-70', 'Age_Number': 65}, {'Subject_Number': 'sub-032305', 'prepSignal_EC': <RawEEGLAB | sub-032305_EC.fdt, 61 x 116012 (464.0 s), ~54.1 MB, data loaded>, 'Age': '65-70', 'Age_Number': 65}, {'Subject_Number': 'sub-032306', 'prepSignal_EC': <RawEEGLAB | sub-032306_EC.fdt, 61 x 119980 (479.9 s), ~55.9 MB, data loaded>, 'Age': '65-70', 'Age_Number': 65}, {'Subject_Number': 'sub-032308', 'prepSignal_EC': <RawEEGLAB | sub-032308_EC.fdt, 61 x 120000 (480.0 s), ~55.9 MB, data loaded>, 'Age': '60-65', 'Age_Number': 60}, {'Subject_Number': 'sub-032318', 'prepSignal_EC': <RawEEGLAB | sub-032318_EC.fdt, 61 x 114791 (459.2 s), ~53.5 MB, data loaded>, 'Age': '60-65', 'Age_Number': 60}, {'Subject_Numbe

In [7]:
print("------------------------------------------------")
print("Splite EEG to epoches")
EEG_epoches = []
for i in tqdm(range(len(EEG))):
    EEG_epoches.append(splitEEG(EEG[i]['prepSignal_EC']))

------------------------------------------------
Splite EEG to epoches


100%|███████████████████████████████████████████| 61/61 [00:01<00:00, 36.99it/s]


In [9]:
len(EEG)

61

In [22]:
EEG[0]['prepSignal_EC'].get_data().shape

(61, 119345)

In [53]:
# print(EEG_epoches)
# Calculate EEG functional connectivity
metric_Name = 'pli'

print("------------------------------------------------")
print("Calculate EEG functional connectivity")

for i, d in enumerate(tqdm(EEG_epoches)):
    FC = EpochRotate(d, metric_Name)
    np.save('/workspace/brail_poc3/experiments/FC_Result_231012/{}/{}.npy'.format(metric_Name, EEG[i]['Subject_Number']), FC)


------------------------------------------------
Calculate EEG functional connectivity


  0%|                                                    | 0/61 [00:00<?, ?it/s]

Using data from preloaded Raw for 265 events and 501 original time points ...
(265, 61, 501)
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])
([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])
([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

([19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])
([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])
([21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 

([37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])
([38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])
([39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 

...

  2%|▋                                        | 1/61 [03:09<3:09:20, 189.34s/it]

Using data from preloaded Raw for 264 events and 501 original time points ...
0 bad epochs dropped
(264, 61, 501)
([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])
([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])
([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2

([19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])
([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60])
([21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 

  2%|▋                                        | 1/61 [04:47<4:47:39, 287.65s/it]


KeyboardInterrupt: 

In [18]:
len(glob.glob("/workspace/brail_poc3/experiments/FC_Result_231102/pli/*"))

201

In [19]:
len(glob.glob("/workspace/brail_poc3/experiments/FC_Result_231102/plv/*"))

201

In [20]:
len(glob.glob("/workspace/brail_poc3/experiments/FC_Result_231102/coh/*"))

201